In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import cv2
import os
import random

In [3]:
from tensorflow import keras
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.layers import Dense, Input
from keras.src.layers import LSTM, Reshape
from keras.src.models import Sequential
from keras.src.layers import BatchNormalization, Activation
from keras.src.optimizers import Adam
from keras.src.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.src.utils import plot_model

%matplotlib inline

In [4]:
img_size = 48
batch_size = 64
sequence_length = 5

In [5]:
def load_dataset(dataset_path, img_size=48, color_mode='grayscale'):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(img_size, img_size),
        color_mode=color_mode,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    return generator

In [6]:
def generate_sequence_data(generator, sequence_length):
    while True:
        batch_x, batch_y = next(generator)
        sequence_x = []
        sequence_y = []
        for i in range(len(batch_x) - sequence_length + 1):
            sequence_x.append(batch_x[i:i + sequence_length])
            sequence_y.append(batch_y[i + sequence_length - 1])
        yield np.array(sequence_x), np.array(sequence_y)

In [8]:
fer_train_generator = load_dataset('fer_train/')
fer_validation_generator = load_dataset('fer_test/')
raf_train_generator = load_dataset('raf_train/')
raf_validation_generator = load_dataset('raf_test/')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Found 12253 images belonging to 7 classes.
Found 3060 images belonging to 7 classes.


In [11]:
train_sequence_generator = generate_sequence_data(fer_train_generator, sequence_length)
validation_sequence_generator = generate_sequence_data(fer_validation_generator, sequence_length)
raf_train_sequence_generator = generate_sequence_data(raf_train_generator, sequence_length)
raf_validation_sequence_generator = generate_sequence_data(raf_validation_generator, sequence_length)


In [12]:
def combined_generator(generator1, generator2):
    while True:
        batch_x1, batch_y1 = next(generator1)
        batch_x2, batch_y2 = next(generator2)
        batch_x = np.concatenate((batch_x1, batch_x2), axis=0)
        batch_y = np.concatenate((batch_y1, batch_y2), axis=0)
        yield batch_x, batch_y

In [13]:
combined_train_generator = combined_generator(train_sequence_generator, raf_train_sequence_generator)
combined_validation_generator = combined_generator(validation_sequence_generator, raf_validation_sequence_generator)


In [14]:
def create_sequential_model(input_shape, sequence_length):
    model = Sequential()
    model.add(Reshape((sequence_length, -1), input_shape=(sequence_length,) + input_shape))
    model.add(LSTM(units=64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(units=256, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

In [15]:
input_shape = (img_size, img_size, 1)
sequential_model = create_sequential_model(input_shape, sequence_length)
sequential_model.summary()

C:\Users\SHAIK\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 5, 2304)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 5, 64)          │       606,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 5, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,236,487 (4.72 MB)

 Trainable params: 1,235,591 (4.71 MB)

 Non-trainable params: 896 (3.50 KB)

In [16]:
epochs = 20
steps_per_epoch = (fer_train_generator.n + raf_train_generator.n) // batch_size
validation_steps = (fer_validation_generator.n + raf_validation_generator.n) // batch_size


In [18]:
checkpoint = ModelCheckpoint("context_model_weights.weights.h5", monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)
callbacks = [checkpoint, reduce_lr]

In [19]:
history = sequential_model.fit(
    combined_train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=combined_validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - accuracy: 0.3704 - loss: 1.6219
Epoch 1: saving model to model_weights.weights.h5
640/640 ━━━━━━━━━━━━━━━━━━━━ 691s 1s/step - accuracy: 0.3705 - loss: 1.6218 - val_accuracy: 0.3860 - val_loss: 1.6592 - learning_rate: 0.0010
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.4881 - loss: 1.3429
Epoch 2: saving model to model_weights.weights.h5
640/640 ━━━━━━━━━━━━━━━━━━━━ 166s 259ms/step - accuracy: 0.4881 - loss: 1.3429 - val_accuracy: 0.2450 - val_loss: 1.8275 - learning_rate: 0.0010
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.5088 - loss: 1.2922
Epoch 3: saving model to model_weights.weights.h5
640/640 ━━━━━━━━━━━━━━━━━━━━ 121s 190ms/step - accuracy: 0.5088 - loss: 1.2921 - val_accuracy: 0.2804 - val_loss: 1.7851 - learning_rate: 0.0010
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5264 - loss: 1.2472
Epoch 4: saving model to model_weights.weights.h5
640/640 ━━━━━━━━

In [21]:
evaluation_results = sequential_model.evaluate(combined_validation_generator, steps=validation_steps)
print(f"Test accuracy: {evaluation_results[1]:.4f}")

159/159 ━━━━━━━━━━━━━━━━━━━━ 19s 119ms/step - accuracy: 0.5241 - loss: 1.4641
Test accuracy: 0.5268


In [22]:
model_json = sequential_model.to_json()
with open("combined_model.json", "w") as json_file:
    json_file.write(model_json)